In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# **ANÁLISIS**

Importamos el archivo `steam_games.parquet` (archivo creado en el proceso de ETL)

In [ ]:
steam_games = pd.read_parquet('..\\Dataset\\steam_games.parquet')
steam_games

Convertimos las columnas `genres`, `tags` y `specs` a cadenas de texto para realizar el Proceso de Lenguaje Natural

In [ ]:
steam_games['genres'] = steam_games['genres'].apply(lambda lista: ", ".join(lista))
steam_games['tags'] = steam_games['tags'].apply(lambda lista: ", ".join(lista))
steam_games['specs'] = steam_games['specs'].apply(lambda lista: ", ".join(lista))

Concatenamos las tres columnas anteriores en una única columna llamada `description` y las eliminamos. Además, eliminamos la columna `release_date`.

In [ ]:
steam_games['description'] = steam_games['genres'] + ', ' + steam_games['tags'] + ', ' + steam_games['specs']
steam_games.drop(columns=['genres', 'tags', 'specs', 'release_date'], inplace=True)
steam_games

Para realizar el proceso de Machine Learning para la función de recomendación de juegos, utilizaremos la propuesta de trabajo que tiene una relación relación ítem-ítem. El objetivo es tomar un item y, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados

Utilizamos la clase **CountVectorizer** de scikit para convertir las descripciones de la columna en representaciones numéricas.

In [ ]:
# Creamos una instancia de la clase CountVectorizer
vector = CountVectorizer(tokenizer= lambda x: x.split(', '))

# Dividimos cada cadena de descripción en palabras individuales y se crea una matriz de conteo que representa cuántas veces aparece cada género en cada videojuego.
matriz_descripcion = vector.fit_transform(steam_games['description'])

In [ ]:
def recomendacion_juego(id_producto: int):
    '''
    Se ingresa el id de producto (item_id) y retorna una lista con 5 juegos recomendados similares al ingresado (title).
    
    '''
    # Si el id ingresado no se encuentra en la columna de id de la tabla 'steam_games' se le pide al usuario que intente con otro id
    if id_producto not in steam_games['item_id'].values:
        return 'El ID no existe, intente con otro'
    else:
        # buscamos el índice del id ingresado
        index = steam_games.index[steam_games['item_id']==id_producto][0]

        # De la matriz de conteo, tomamos el array de descripciones con índice igual a 'index'
        description_index = matriz_descripcion[index]

        # Calculamos la similitud coseno entre la descripción de entrada y la descripción de las demás filas: cosine_similarity(description_index, matriz_descripcion)
        # Obtenemos los índices de las mayores similitudes mediante el método argsort() y las similitudes ordenadas de manera descendente
        # Tomamos los índices del 1 al 6 [0, 1:6] ya que el índice 0 es el mismo índice de entrada
        indices_maximos = np.argsort(-cosine_similarity(description_index, matriz_descripcion))[0, 1:6]

        # Construimos la lista
        recomendaciones = []
        for i in indices_maximos:
            recomendaciones.append(steam_games['title'][i])
        
        return recomendaciones